# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

%matplotlib notebook

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = '../Output_Data/weatherpy.csv'
df = pd.read_csv(data, low_memory=False)

df.head()

,Unnamed: 0,CityName,Latitude,Longitude,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,Jega,12.22,4.38,72.09,88,100,10.02,NG,1596833027
1,1,Ancud,-41.87,-73.82,51.01,100,75,11.41,CL,1596833048
2,2,Rivadavia,-33.19,-68.46,59.00,24,96,5.82,AR,1596833048
3,3,Bengkulu,-3.80,102.27,76.03,91,85,3.29,ID,1596833049
4,4,Rikitea,-23.12,-134.97,69.71,72,76,18.45,PF,1596833050


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = df[["Latitude", "Longitude"]].astype(float)

In [5]:
humidity = df["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=df.loc[(df['MaxTemp']<80) & (df['MaxTemp']>70) & (df["WindSpeed"]<10) &
               (df["Cloudiness"]==0)]

In [8]:
hotel_df["Hotel Name"] = ""

C:\Users\Familia Roque\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
hotel_df.head()

,Unnamed: 0,CityName,Latitude,Longitude,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
47,47,Aksaray,38.37,34.03,77.00,25,0,9.17,TR,1596833077,
80,80,Abu Samrah,35.30,37.18,73.58,64,0,8.72,SY,1596833098,
140,140,Porto Torres,40.84,8.40,77.00,73,0,5.82,IT,1596833135,
164,164,Lata,40.16,-8.33,73.99,70,0,3.18,PT,1596833150,
199,199,Dawlatabad,36.41,64.91,75.29,32,0,3.40,AF,1596833172,


In [10]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\Familia Roque\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{CityName}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map

fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
hotel_df

,Unnamed: 0,CityName,Latitude,Longitude,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
47,47,Aksaray,38.37,34.03,77.00,25,0,9.17,TR,1596833077,Grand Altuntaş Hotel
80,80,Abu Samrah,35.30,37.18,73.58,64,0,8.72,SY,1596833098,
140,140,Porto Torres,40.84,8.40,77.00,73,0,5.82,IT,1596833135,Hotel Libyssonis
164,164,Lata,40.16,-8.33,73.99,70,0,3.18,PT,1596833150,Magnolias Cottage
199,199,Dawlatabad,36.41,64.91,75.29,32,0,3.40,AF,1596833172,
210,210,Saint-François,46.42,3.91,78.80,36,0,4.34,FR,1596833157,
276,276,Santiago del Estero,-27.80,-64.26,71.56,37,0,7.76,AR,1596832996,Hotel Savoy
327,327,Vila,42.03,-8.16,77.00,69,0,5.82,PT,1596833252,Hotel Castrum Villae
358,358,Presidente Epitácio,-21.76,-52.12,75.94,29,0,5.23,BR,1596833271,Pousada Pesca Tucuna
384,384,Boshkengash,38.47,68.81,75.20,47,0,6.71,TJ,1596833286,
